# Logistic Regression

### Weronica - ML Goal: Test how well logisitc regression performance on our classification task and time performance on a 4XL EMR Cluster. 

In [1]:
%%configure -f 
{
"conf":{
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
        "spark.executor.heartbeatInterval":"10800s",
        "spark.network.timeout":"24h",
        "spark.driver.memory": "40G",
        "spark.executor.memory": "40G",
        "spark.executor.cores":"10",
        "livy.server.session.timeout":"24h",
        "spark.app.name":"preproccessing_please_work_2"
      }
}

In [2]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, size, split
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import time

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1615410459390_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
final_df=spark.read.parquet("s3a://facebookmsds694/final_df.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
muliticlass_df = final_df.withColumn("num_labels",size(split(col('Tags')," ").cast("array<string>"))).filter(col("num_labels")==1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
tags = muliticlass_df.groupBy('Tags_joined').count()
tags.show(20, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-----+
|Tags_joined            |count|
+-----------------------+-----+
|qt                     |1328 |
|algebra-precalculus    |2294 |
|google-chrome-extension|715  |
|windows-7              |2674 |
|magento                |3710 |
|c#-3.0                 |384  |
|iphone                 |12552|
|geometry               |2007 |
|unix                   |805  |
|database-design        |737  |
|ssis                   |721  |
|windows-server-2008    |714  |
|zend-framework         |1077 |
|asp.net-mvc-2          |821  |
|ftp                    |362  |
|xcode                  |1291 |
|azure                  |639  |
|android                |49783|
|virtualbox             |407  |
|macros                 |455  |
+-----------------------+-----+
only showing top 20 rows

In [6]:
tags.show(muliticlass_df.count(), truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+-----+
|Tags_joined              |count|
+-------------------------+-----+
|qt                       |1328 |
|algebra-precalculus      |2294 |
|google-chrome-extension  |715  |
|magento                  |3710 |
|windows-7                |2674 |
|c#-3.0                   |384  |
|iphone                   |12552|
|ssis                     |721  |
|geometry                 |2007 |
|unix                     |805  |
|database-design          |737  |
|windows-server-2008      |714  |
|zend-framework           |1077 |
|asp.net-mvc-2            |821  |
|ftp                      |362  |
|xcode                    |1291 |
|azure                    |639  |
|android                  |49783|
|virtualbox               |407  |
|jsf-2                    |443  |
|macros                   |455  |
|infopath                 |384  |
|notation                 |516  |
|algorithm                |1838 |
|entity-framework-4       |371  |
|couchdb                  |390  |
|applescript  

In [7]:
tags.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

345

In [8]:
muliticlass_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

617133

In [9]:
c='Tags'
si = StringIndexer(inputCol=muliticlass_df.columns[3], outputCol=c + "-num")
sm = si.fit(muliticlass_df)
newdf = sm.transform(muliticlass_df).drop(c)
newdf = newdf.withColumnRenamed(c + "-num", c)
final_df.unpersist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Text_array: string, Title: string, Body: string, Tags: string, Text_array_new: array<string>, Text_Vectors: vector, Tags_joined: string, count: bigint]

In [10]:
newdf.select('Text_Vectors', 'Tags').show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+
|        Text_Vectors|Tags|
+--------------------+----+
|(20000,[19,20,30,...|25.0|
|(20000,[2,66,73,9...|35.0|
|(20000,[0,1,6,41,...|39.0|
+--------------------+----+
only showing top 3 rows

In [11]:
splits = newdf.randomSplit([0.8, 0.2], seed=1)
train=splits[0].cache()
test=splits[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
train=train.select(col("Text_Vectors").alias("features"),col("Tags").alias("label")).cache()
test=test.select(col("Text_Vectors").alias("features"),col("Tags").alias("label")).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Logisitic Regression Performance

In [14]:
start = time.time()

lr = LogisticRegression(maxIter=5, regParam=0.3, elasticNetParam=0)
lr_model = lr.fit(train)

model_predictions = lr_model.transform(test)

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
accuracy = evaluator.evaluate(model_predictions)
print("Accuracy = %g" % accuracy)

print('Time Performance: ', time.time()-start, 'seconds.')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.440775
Time Performance:  34.879525661468506 seconds.

### Conclusion: 
- Accuracy = 0.440775
- 2XL Cluster Performance about 35 Seconds